## Generate report (flow, speed, and V/C) for links with a given functional class

In [ ]:
# Highway links report (flow, V/C, and speeds) by functional class
#
import openmatrix as omx
import numpy as np
import pandas as pd
import geopandas as gp
import matplotlib.pyplot as plt

In [ ]:
# Base directory for MoDX output for "base year" model results.
#
base_scenario_dir = r'G:/Regional_Modeling/1A_Archives/LRTP_2018/2016 Scen 00_08March2019_MoDXoutputs/'
#
# Base directory for MoDX output for "comparison scenario" model results.
# 
comparison_scenario_dir = r'G:/Regional_Modeling/1A_Archives/LRTP_2018/2040 NB Scen 01_MoDXoutputs/'
# 

### User input required: Path to root directory for current scenario

In [ ]:
# ===>>>USER INPUT REQUIRED: <<<===
#
# 1. Supply path to root directory of scenario to use for the current run of this notebook:
# 
home_dir = base_scenario_dir

### User input required: Path to root of user's "sandbox" directory

In [ ]:
# 2. Supply path to root of user's "sandbox" directory:
#
my_sandbox_dir = r'S:/my_modx_output_dir/'

### User input required: CSV file for report output

In [ ]:
# 3. Supply name of CSV output file for tabular results generated by this notebook:
#
csv_output_fn = 'functional_class_report_base_scenario.csv'

### User input required: Python list of functional class number(s)

In [ ]:
# 4. Supply array of selected functional class number(s).
#    NOTE: Even if a report for a _single_ functionl class is desired, this value must be supplied in a list!
#
selected_fc_list = [1, 2]

In [ ]:
# Directory in which the spatial data for the model network links is stored (both shapefile and GeoJSON formats)
links_spatial_data_dir = r'G:/Data_Resources/modx/statewide_links_shapefile/'

In [ ]:
# Load the links shapefile into a geopandas dataframe 
links_shapefile_fn = 'Statewide_Links_2018_BK_EPSG26986.shp'
fq_links_shapefile_fn = links_spatial_data_dir + links_shapefile_fn
links_gdf = gp.read_file(fq_links_shapefile_fn)
links_gdf.set_index("ID")

In [ ]:
# Get the list of links with the specified functional clas(ses).
# NOTE: The attribute to query is 'SCEN_00_FU', not the most felicitous choice of attribute name!
#
filtered_gdf = links_gdf[links_gdf['SCEN_00_FU'].isin(selected_fc_list)]

In [ ]:
# Get the list of IDs for the links with the specified functional clas(ses).
highway_links_list = filtered_gdf['ID'].to_list()

In [ ]:
# Directory containing link flow CSVs
link_flow_dir = home_dir + 'out/'

In [ ]:
# Individual link-flow CSV tables:
# For each time period, there is a separate flow CSV for autos and for trucks.
# To get the total volume for any given time period, 'Tot_Flow' columns these need to be summed.
# However, the V/C and speed data for *both* autos and trucks are reported in the CSV for autos.
# Clear?
#
am_flow_auto_fn = link_flow_dir + 'AM_MMA_LinkFlow.csv'
am_flow_truck_fn = link_flow_dir + 'AM_MMA_LinkFlow_Trucks.csv'
#
md_flow_auto_fn = link_flow_dir + 'MD_MMA_LinkFlow.csv'
md_flow_truck_fn = link_flow_dir + 'MD_MMA_LinkFlow_Trucks.csv'
#
pm_flow_auto_fn = link_flow_dir + 'PM_MMA_LinkFlow.csv'
pm_flow_truck_fn = link_flow_dir + 'PM_MMA_LinkFlow_Trucks.csv'
#
nt_flow_auto_fn = link_flow_dir + 'NT_MMA_LinkFlow.csv'
nt_flow_truck_fn = link_flow_dir + 'NT_MMA_LinkFlow_Trucks.csv'

In [ ]:
# Read each of the above CSV files containing flow data into a dataframe
#
temp_am_auto_df = pd.read_csv(am_flow_auto_fn, delimiter=',')
temp_am_truck_df = pd.read_csv(am_flow_truck_fn, delimiter=',')
#
temp_md_auto_df = pd.read_csv(md_flow_auto_fn, delimiter=',')
temp_md_truck_df = pd.read_csv(md_flow_truck_fn, delimiter=',')
#
temp_pm_auto_df = pd.read_csv(pm_flow_auto_fn, delimiter=',')
temp_pm_truck_df = pd.read_csv(pm_flow_truck_fn, delimiter=',')
#
temp_nt_auto_df = pd.read_csv(nt_flow_auto_fn, delimiter=',')
temp_nt_truck_df = pd.read_csv(nt_flow_truck_fn, delimiter=',') 

In [ ]:
# Filter the 8 temp "flow" dataframes to only include rows for the selected highway links
#
am_auto_df = temp_am_auto_df[temp_am_auto_df['ID1'].isin(highway_links_list)]
am_truck_df = temp_am_truck_df[temp_am_truck_df['ID1'].isin(highway_links_list)]
#
md_auto_df = temp_md_auto_df[temp_md_auto_df['ID1'].isin(highway_links_list)]
md_truck_df = temp_md_truck_df[temp_md_truck_df['ID1'].isin(highway_links_list)]
#
pm_auto_df = temp_pm_auto_df[temp_pm_auto_df['ID1'].isin(highway_links_list)]
pm_truck_df = temp_pm_truck_df[temp_pm_truck_df['ID1'].isin(highway_links_list)]
#
nt_auto_df = temp_nt_auto_df[temp_nt_auto_df['ID1'].isin(highway_links_list)]
nt_truck_df = temp_nt_truck_df[temp_nt_truck_df['ID1'].isin(highway_links_list)]
#
# NOTE: volume/capacity and speed data will be harvested from the "auto" dataframes subsequently.
#       See below.

In [ ]:
# Further filter the filetered "flow" datafames to only include the columns containing 'Tot_Flow' (and 'ID1' and 'SCEN_00_FU')
# 
am_auto_vol_df = am_auto_df[['ID1', 'Tot_Flow']]
am_truck_vol_df = am_truck_df[['ID1', 'Tot_Flow']]
#
md_auto_vol_df = md_auto_df[['ID1', 'Tot_Flow']]
md_truck_vol_df = md_truck_df[['ID1', 'Tot_Flow']]
#
pm_auto_vol_df = pm_auto_df[['ID1', 'Tot_Flow']]
pm_truck_vol_df = pm_truck_df[['ID1', 'Tot_Flow']]
#
nt_auto_vol_df = nt_auto_df[['ID1', 'Tot_Flow']]
nt_truck_vol_df = nt_truck_df[['ID1', 'Tot_Flow']]

In [ ]:
# Rename the 'Tot_Flow' column of each dataframe, appropriately
#
am_auto_vol_df = am_auto_vol_df.rename(columns={'Tot_Flow' : 'Tot_Flow_am_auto'})
am_truck_vol_df = am_truck_vol_df.rename(columns={'Tot_Flow' : 'Tot_Flow_am_truck'})
#
md_auto_vol_df = md_auto_vol_df.rename(columns={'Tot_Flow' : 'Tot_Flow_md_auto'})
md_truck_vol_df = md_truck_vol_df.rename(columns={'Tot_Flow' : 'Tot_Flow_md_truck'})
#
pm_auto_vol_df = pm_auto_vol_df.rename(columns={'Tot_Flow' : 'Tot_Flow_pm_auto'})
pm_truck_vol_df = pm_truck_vol_df.rename(columns={'Tot_Flow' : 'Tot_Flow_pm_truck'})
#
nt_auto_vol_df = nt_auto_vol_df.rename(columns={'Tot_Flow' : 'Tot_Flow_nt_auto'})
nt_truck_vol_df = nt_truck_vol_df.rename(columns={'Tot_Flow' : 'Tot_Flow_nt_truck'})

In [ ]:
# Index all the "volume" dataframes on "ID1", in preparation for joining
#
am_auto_vol_df.set_index("ID1")
am_truck_vol_df.set_index("ID1")
#
md_auto_vol_df.set_index("ID1")
md_truck_vol_df.set_index("ID1")
#
pm_auto_vol_df.set_index("ID1")
pm_truck_vol_df.set_index("ID1")
#
nt_auto_vol_df.set_index("ID1")
nt_truck_vol_df.set_index("ID1")

In [ ]:
# Join the "volume" dataframes
j1_df = am_auto_vol_df.join(am_truck_vol_df.set_index("ID1"), on="ID1")
#
j1_df.set_index("ID1")
j2_df = j1_df.join(md_auto_vol_df.set_index("ID1"), on="ID1")
#
j2_df.set_index("ID1")
j3_df = j2_df.join(md_truck_vol_df.set_index("ID1"), on="ID1")
#
j3_df.set_index("ID1")
j4_df = j3_df.join(pm_auto_vol_df.set_index("ID1"), on="ID1")
#
j4_df.set_index("ID1")
j5_df = j4_df.join(pm_truck_vol_df.set_index("ID1"), on="ID1")
#
j5_df.set_index("ID1")
j6_df = j5_df.join(nt_auto_vol_df.set_index("ID1"), on="ID1")
#
j6_df.set_index("ID1")
total_flow_join = j6_df.join(nt_truck_vol_df.set_index("ID1"), on="ID1")
#
total_flow_join.set_index("ID1")

In [ ]:
# Calculate the total volume (auto + truck) for each time period, and for the entire day
#
total_flow_join['Tot_Flow_am'] = total_flow_join['Tot_Flow_am_auto'] + total_flow_join['Tot_Flow_am_truck']
#
total_flow_join['Tot_Flow_md'] = total_flow_join['Tot_Flow_md_auto'] + total_flow_join['Tot_Flow_md_truck']
#
total_flow_join['Tot_Flow_pm'] = total_flow_join['Tot_Flow_pm_auto'] + total_flow_join['Tot_Flow_pm_truck']
#
total_flow_join['Tot_Flow_nt'] = total_flow_join['Tot_Flow_nt_auto'] + total_flow_join['Tot_Flow_nt_truck']
#
total_flow_join['Tot_Flow_daily'] = total_flow_join['Tot_Flow_am'] + total_flow_join['Tot_Flow_md'] + \
                                    total_flow_join['Tot_Flow_pm'] + total_flow_join['Tot_Flow_nt']

In [ ]:
# Sanity check
total_flow_join

In [ ]:
total_flow_join.set_index("ID1")

In [ ]:
# Harvest the speed and volume-to-capacity ratio data from the 4 "auto" dataframes, one for each time period. (See above.)
# Note Python variable naming convention used here: "svc" == "speed and volume/capacity"
#
am_svc_df = am_auto_df[['ID1', 'AB_Speed', 'BA_Speed', 'AB_VOC', 'BA_VOC']]
#
md_svc_df = md_auto_df[['ID1', 'AB_Speed', 'BA_Speed', 'AB_VOC', 'BA_VOC']]
#
pm_svc_df = pm_auto_df[['ID1', 'AB_Speed', 'BA_Speed', 'AB_VOC', 'BA_VOC']]
#
nt_svc_df = nt_auto_df[['ID1', 'AB_Speed', 'BA_Speed', 'AB_VOC', 'BA_VOC']]

In [ ]:
# Sanity check
am_svc_df

In [ ]:
# Rename the columns of these "svc" dataframes in preparation for joining them with the speed dataframe, computed above.
#
am_svc_df = am_svc_df.rename(columns={'AB_Speed' : 'AB_Speed_am', 
                                      'BA_Speed' : 'BA_Speed_am',
                                      'AB_VOC'   : 'AB_VOC_am', 
                                      'BA_VOC'   : 'BA_VOC_am'})
#
md_svc_df = md_svc_df.rename(columns={'AB_Speed' : 'AB_Speed_md', 
                                      'BA_Speed' : 'BA_Speed_md',
                                      'AB_VOC'   : 'AB_VOC_md', 
                                      'BA_VOC'   : 'BA_VOC_md'})
#
pm_svc_df = pm_svc_df.rename(columns={'AB_Speed' : 'AB_Speed_pm', 
                                      'BA_Speed' : 'BA_Speed_pm',
                                      'AB_VOC'   : 'AB_VOC_pm', 
                                      'BA_VOC'   : 'BA_VOC_pm'})
#
nt_svc_df = nt_svc_df.rename(columns={'AB_Speed' : 'AB_Speed_nt', 
                                      'BA_Speed' : 'BA_Speed_nt',
                                      'AB_VOC'   : 'AB_VOC_nt', 
                                      'BA_VOC'   : 'BA_VOC_nt'})

In [ ]:
# Sanity check
nt_svc_df

In [ ]:
# Per instructions from Marty on June 22, 2021:
# For a given time period, calculate the MIN of the AB_Speed and BA_Speed, and the MAX of the AB_VOC and BA_VOC.
# Basically, the idea is to flag the link direction with the most congestion.
#
am_svc_df['Speed_am'] = am_svc_df.apply(lambda x: min(x['AB_Speed_am'], x['BA_Speed_am']), axis=1)
am_svc_df['VOC_am'] = am_svc_df.apply(lambda x: max(x['AB_VOC_am'], x['BA_VOC_am']), axis=1)
#
md_svc_df['Speed_md'] = md_svc_df.apply(lambda x: min(x['AB_Speed_md'], x['BA_Speed_md']), axis=1)
md_svc_df['VOC_md'] = md_svc_df.apply(lambda x: max(x['AB_VOC_md'], x['BA_VOC_md']), axis=1)
#
pm_svc_df['Speed_pm'] = pm_svc_df.apply(lambda x: min(x['AB_Speed_pm'], x['BA_Speed_pm']), axis=1)
pm_svc_df['VOC_pm'] = pm_svc_df.apply(lambda x: max(x['AB_VOC_pm'], x['BA_VOC_pm']), axis=1)
#
nt_svc_df['Speed_nt'] = nt_svc_df.apply(lambda x: min(x['AB_Speed_nt'], x['BA_Speed_nt']), axis=1)
nt_svc_df['VOC_nt'] = nt_svc_df.apply(lambda x: max(x['AB_VOC_nt'], x['BA_VOC_nt']), axis=1)

In [ ]:
# Santiy check #1
am_svc_df

In [ ]:
# Santiy check #2
md_svc_df

In [ ]:
# Santiy check #3
pm_svc_df

In [ ]:
# Santiy check #4
nt_svc_df

In [ ]:
# Index the "svc" dataframes in preparation for joining
am_svc_df.set_index("ID1")
md_svc_df.set_index("ID1")
pm_svc_df.set_index("ID1")
nt_svc_df.set_index("ID1")

In [ ]:
# Join the speed and volume/capacity data to the volume data collected above into a single dataframe.
#
j7_df = total_flow_join.join(am_svc_df.set_index("ID1"), on="ID1")
j7_df.set_index("ID1")
#
j8_df = j7_df.join(md_svc_df.set_index("ID1"), on="ID1")
j8_df.set_index("ID1")
#
j9_df = j8_df.join(pm_svc_df.set_index("ID1"), on="ID1")
j9_df.set_index("ID1")
#
all_data_df = j9_df.join(nt_svc_df.set_index("ID1"), on="ID1")
all_data_df.set_index("ID1")

### Export report output

In [ ]:
# Export the dataframe as a CSV file
fq_output_fn = my_sandbox_dir + csv_output_fn
all_data_df.to_csv(fq_output_fn, sep=',')